In [1]:
import matplotlib as plt
import numpy as np
from PIL import Image
import time as tm
import pandas as pd
import copy as cpy
import sys

In [2]:
class NN_DigitsClassify:
    def __init__(self, hidden_dim):
        self.W1 = (np.random.randn(65,hidden_dim))/8
        self.W2 = (np.random.randn(hidden_dim+1, 10))/np.sqrt(hidden_dim+1)
        
    def one_epoch(self,X,y,alpha,lambda1):
        #forward propagation
        a1 = np.append(np.ones([len(X),1]),X,1)
        z1 = np.dot(a1,self.W1)
        exp_z1 = np.exp((-1)*z1)
        x2 = (1.)/(1 + exp_z1)
        a2 = np.append(np.ones([len(x2),1]),x2,1)
        z2 = np.dot(a2,self.W2)
        
        exp_z2 = np.exp(z2)
        softmax_scores = exp_z2 / np.sum(exp_z2, axis=1, keepdims=True)
        
        # Backpropagation
        one_hot_y = np.zeros_like(softmax_scores)
        for i in range(X.shape[0]):
            one_hot_y[i,y[i]] = 1
        beta3 =  softmax_scores - one_hot_y
        
        gdash2 = np.multiply(x2,(1-x2))
        beta2 = np.multiply(np.dot(beta3, np.transpose(self.W2[1:,:])) , gdash2)
        #Update weights
        updW2 = alpha * (np.dot(np.transpose(a2),beta3) + lambda1*self.W2)
        updW1 = alpha * (np.dot(np.transpose(a1),beta2) + lambda1*self.W1)
        self.W2 = self.W2 - updW2
        self.W1 = self.W1 - updW1
    
    def fit(self,X,y,num_epochs,alpha=0.01,lambda1=0):
        #print(self.W1, self.W2)
        for epoch in range(0, num_epochs):
            self.one_epoch(X,y,alpha,lambda1)
            
    def predict(self,X):
        a1 = np.append(np.ones([len(X),1]),X,1)
        z1 = np.dot(a1,self.W1)
        exp_z1 = np.exp((-1)*z1)
        x2 = (1.)/(1 + exp_z1)
        a2 = np.append(np.ones([len(x2),1]),x2,1)
        z2 = np.dot(a2,self.W2)        
        exp_z2 = np.exp(z2)
        softmax_scores = exp_z2 / np.sum(exp_z2, axis=1, keepdims=True)
        predictions = np.argmax(softmax_scores, axis = 1)
        return predictions

In [3]:
X_train = np.genfromtxt('/Users/pavithraraghavan/Downloads/CS440640_Lab5/CS440640_Lab5/DATA/Digit_X_train.csv', delimiter=',')
y_train = np.genfromtxt('/Users/pavithraraghavan/Downloads/CS440640_Lab5/CS440640_Lab5/DATA/Digit_y_train.csv', delimiter=',').astype(np.int64)

In [4]:
#Initialize model
#input_dim = np.shape(X)[1]
#output_dim = np.max(y) + 1
hidden_dim = 50
NN = NN_DigitsClassify(hidden_dim)

In [5]:
NN.fit(X_train,y_train,5000,alpha=0.01, lambda1=0.03)

In [6]:
X_test = np.genfromtxt('/Users/pavithraraghavan/Downloads/CS440640_Lab5/CS440640_Lab5/DATA/Digit_X_test.csv', delimiter=',')
y_test = np.genfromtxt('/Users/pavithraraghavan/Downloads/CS440640_Lab5/CS440640_Lab5/DATA/Digit_y_test.csv', delimiter=',').astype(np.int64)

In [8]:
#Accuracy on training data
y_pred1 = NN.predict(X_train)
acc = 0
con_mat = np.zeros((10, 10))
for i in range(len(y_pred1)):
    con_mat[y_pred1[i], y_train[i]] += 1
    if y_train[i] == y_pred1[i]:
        acc += 1.0
#print(acc)
#print(np.sum(y_pred==y))
acc = acc/len(y_pred1)
print ('ACCURACY: ', acc)
print ('CONFUSION MATRIX: ')
print(con_mat)

('ACCURACY: ', 1.0)
CONFUSION MATRIX: 
[[90.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 91.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. 91.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 92.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 89.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 91.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 90.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. 90.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. 86.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. 88.]]


In [9]:
#Accuracy on testing data
acc = 0
y_pred2 = NN.predict(X_test)
con_mat = np.zeros((10, 10))
for i in range(len(y_pred2)):
    con_mat[y_pred2[i], y_test[i]] += 1
    if y_test[i] == y_pred2[i]:
        acc += 1.0
acc = acc/len(y_pred2)
print ('ACCURACY: ', acc)
print ('CONFUSION MATRIX: ')
print(con_mat)

('ACCURACY: ', 0.9354838709677419)
CONFUSION MATRIX: 
[[85.  2.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 82.  0.  1.  1.  1.  1.  0.  2.  0.]
 [ 0.  0. 85.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1. 77.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0. 85.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  3.  0. 84.  0.  1.  2.  2.]
 [ 2.  1.  0.  0.  3.  2. 90.  0.  0.  0.]
 [ 0.  0.  0.  2.  0.  0.  0. 84.  1.  0.]
 [ 0.  1.  0.  7.  1.  1.  0.  0. 82.  2.]
 [ 0.  5.  0.  1.  2.  3.  0.  3.  1. 87.]]


In [38]:
#display the input image

image1 = np.zeros([20,8,8])
x = np.zeros([20,8,152])
for j in range(0, 10):
    for i in range(0, 10):
        data = np.reshape(255*X_test[i],(8,8)) #i--the ith digit
        image1[i*2] = Image.fromarray(data)
    x[j*2] = np.hstack((image1[0:19]))
#data = np.reshape(255*X_train[2],(8,8)) #i--the ith digit
#image2 = Image.fromarray(data)

x = np.vstack((x[0:19]))
x = Image.fromarray(x)
#x.save('/Users/pavithraraghavan/Documents/BU/2nd_sem/AI/P1/Plots/digits_test_in.jpg')
x.show()

In [27]:
y_predicted = np.zeros([10,10])
for j in range(0, 10):
    for i in range(0, 10):
        y_predicted[j,i] = y_pred2[(j*10)+i]
        #print(y_pred2[(j*10)+i])
print(y_predicted)

[[8. 8. 4. 9. 0. 8. 9. 8. 1. 2.]
 [3. 4. 5. 6. 7. 8. 9. 0. 1. 2.]
 [3. 4. 5. 6. 7. 8. 9. 0. 1. 2.]
 [3. 4. 9. 6. 7. 8. 9. 0. 9. 5.]
 [5. 6. 5. 0. 9. 8. 9. 8. 4. 1.]
 [7. 7. 3. 9. 1. 2. 7. 8. 2. 0.]
 [1. 2. 6. 3. 3. 7. 3. 3. 4. 6.]
 [6. 6. 4. 9. 1. 5. 0. 9. 5. 2.]
 [8. 2. 0. 0. 1. 7. 6. 3. 2. 1.]
 [4. 6. 3. 1. 3. 9. 1. 7. 6. 8.]]


In [ ]:
#export matrix to csv
pd.set_option('display.precision',5)
df =pd.DataFrame(y_predicted)
df.style
print(df)
df.to_csv('/Users/pavithraraghavan/Documents/BU/2nd_sem/AI/P1/Plots/digits_predicted.csv', encoding='utf-8', header=False, index=False)

     0    1    2    3    4    5    6    7    8    9
0  8.0  8.0  4.0  9.0  0.0  8.0  9.0  8.0  1.0  2.0
1  3.0  4.0  5.0  6.0  7.0  8.0  9.0  0.0  1.0  2.0
2  3.0  4.0  5.0  6.0  7.0  8.0  9.0  0.0  1.0  2.0
3  3.0  4.0  9.0  6.0  7.0  8.0  9.0  0.0  9.0  5.0
4  5.0  6.0  5.0  0.0  9.0  8.0  9.0  8.0  4.0  1.0
5  7.0  7.0  3.0  9.0  1.0  2.0  7.0  8.0  2.0  0.0
6  1.0  2.0  6.0  3.0  3.0  7.0  3.0  3.0  4.0  6.0
7  6.0  6.0  4.0  9.0  1.0  5.0  0.0  9.0  5.0  2.0
8  8.0  2.0  0.0  0.0  1.0  7.0  6.0  3.0  2.0  1.0
9  4.0  6.0  3.0  1.0  3.0  9.0  1.0  7.0  6.0  8.0


TypeError: to_csv() got an unexpected keyword argument 'Header'